In [1]:
import pandas as pd

In [2]:
df_data = pd.read_csv('A1_dataset.csv')

In [3]:
df_data

,LABEL,DATE_TIME,TEXT
0,0,Fri Jun 05 14:26:50 2009,About to get threaded and scared
1,1,Thu May 14 10:13:55 2009,@awaisnaseer I like Shezan Mangooo too!!! I ha...
2,1,Fri Jun 05 21:02:20 2009,worked on my car after work. showering then go...
3,1,Sun Jun 14 22:25:52 2009,@Marama Actually we start this afternoon! I w...
4,1,Sun May 31 00:42:12 2009,@gfalcone601 Aww Gi.don't worry.we'll vote for...
...,...,...,...
4282,1,Sat Jun 06 22:45:26 2009,@QandQ My performances on my CLEP tests. #qshock
4283,0,Tue Jun 16 10:17:07 2009,"ugh no, rcn had all the true blood episodes on..."
4284,1,Fri May 01 22:00:42 2009,Just returned from the forest! Sarah (my merch...
4285,1,Sun Jun 07 02:09:46 2009,is proud of her dad and his piece of work. ( h...


In [4]:
df_data['TEXT'].to_list()[:10]

['About to get threaded and scared ',
 '@awaisnaseer I like Shezan Mangooo too!!! I had one yesterday ',
 'worked on my car after work. showering then going to bed. sooooooooooo tired. sparrow signing out  &lt;Cowboy Up&gt;',
 '@Marama Actually we start this afternoon!  I will try to have something by Wed. It will be a slow process of collecting, many on vac.',
 "@gfalcone601 Aww Gi.don't worry.we'll vote for you non-stop coz we love you so much ",
 '@mrstessyman What ever you do have a good day. I love knitpicks ',
 '@GetMeVideo Sorry, not my forte,  Ask me about #trading, #scalping the dax and I can answer ',
 "Getting ready for church and bummed I cannot watch Rafa whoop Djokovic LIVE, I'll settle for DVR I suppose ",
 'Up early tomorrow. Last open home. Goodnight ',
 "Needs to shake this gloomy feeling!!    Maybe it's the rain???"]

In [5]:
from customregexes import *

# I. REGULAR EXPRESSION

# A.

## a. Average number of sentences and tokens 
- ! or ? (Even Continous)
- . Followed by one or more spaces and then a capital character

In [6]:
df_data['sentence_count'] = df_data['TEXT'].apply(lambda x: findSentenceCount(x))

In [7]:
df_data.groupby(['LABEL'])['sentence_count'].mean()

LABEL
0    1.803000
1    1.856143
Name: sentence_count, dtype: float64

In [13]:
def findTokens(sentence):
  # Split on Whitespace using regex
  tokens = re.split("\s+", sentence)
  # Check URLs and Users
  allTokens = []
  for token in tokens:
    if findURLs(token):
      URLs = findURLs(token)
      for url in URLs:
        indexes = re.finditer(url, token)
        for index in indexes:
          allTokens.append(token[index.start():index.end()])
          allTokens.append(token[:index.start()])
          allTokens.append(token[index.end():])
    elif findUsernames(token):
      usernames = findUsernames(token)
      for username in usernames:
        indexes = re.finditer(username, token)
        for index in indexes:
          allTokens.append(token[index.start():index.end()])
          allTokens.append(token[:index.start()])
          allTokens.append(token[index.end():])
    else:
      # split on punctuation
      if re.search("[\.!?]+", token):
        allTokens.extend(re.split("([\.!?]+)", token))
      else:
        allTokens.append(token)
  # Remove empty strings
  new_token_components = []
  for i in range(len(allTokens)):
    if not match_empty(allTokens[i]):
      new_token_components.append(allTokens[i])
  # Remove leading and trailing spaces
  allTokens = new_token_components
  for i in range(len(allTokens)):
    allTokens[i] = lrstrip(allTokens[i])
  return allTokens

In [14]:
s = "@awaisnaseer I like Shezan Mangooo too!!! I had one yesterday www.google.com. Helllooo Gi.don't"
findTokens(s)

['@awaisnaseer',
 'I',
 'like',
 'Shezan',
 'Mangooo',
 'too',
 '!!!',
 'I',
 'had',
 'one',
 'yesterday',
 'www.google.com',
 '.',
 'Helllooo',
 'Gi',
 '.',
 "don't"]

In [15]:
df_data['token_count'] = df_data['TEXT'].apply(lambda x: findTokenCount(x))

In [16]:
df_data['tokens'] = df_data['TEXT'].apply(lambda x: findTokens(x))

In [17]:
df_data.groupby(['LABEL'])['token_count'].mean()

LABEL
0    13.605500
1    12.829908
Name: token_count, dtype: float64

In [18]:
df_data[['TEXT', 'tokens']].to_csv('A1_dataset_with_tokens.csv')

,TEXT,tokens
0,About to get threaded and scared,"[About, to, get, threaded, and, scared]"
1,@awaisnaseer I like Shezan Mangooo too!!! I ha...,"[@awaisnaseer, I, like, Shezan, Mangooo, too, ..."
2,worked on my car after work. showering then go...,"[worked, on, my, car, after, work, ., showerin..."
3,@Marama Actually we start this afternoon! I w...,"[@Marama, Actually, we, start, this, afternoon..."
4,@gfalcone601 Aww Gi.don't worry.we'll vote for...,"[@gfalcone601, Aww, Gi, ., don't, worry, ., we..."
...,...,...
4282,@QandQ My performances on my CLEP tests. #qshock,"[@QandQ, My, performances, on, my, CLEP, tests..."
4283,"ugh no, rcn had all the true blood episodes on...","[ugh, no,, rcn, had, all, the, true, blood, ep..."
4284,Just returned from the forest! Sarah (my merch...,"[Just, returned, from, the, forest, !, Sarah, ..."
4285,is proud of her dad and his piece of work. ( h...,"[is, proud, of, her, dad, and, his, piece, of,..."


## b. Words Starting With Consonants or Vowels

Basic Rules and Assumptions:

- Word Boundary or Punctuation then a Vowel followed by only Alphabets (accented-non-accented both) and nothing else so A9A is not a word

- Word Boundary or Punctuation then a Consonant followed by only Alphabets (accented-non-accented both) and nothing else so A9A is not a word

- Compound Words like truck-driver can also be there

- Since it is Twitter data the data can be multilingual/can have non ascii characters (stuff like accented characters) and they can be handled.

In [65]:
def findWordsStartingWithVowel(sentence):
  return len(re.findall("\\b[aeiouAEIOU][\w|À-Ö|Ø-ö|ø-ÿ|'|-]*\\b", sentence))

In [67]:
findWordsStartingWithVowel("Aww Gi...Aon't worry:ae'll \n vote for UUUU and IM. A9A ae-l ee:ef aruck-ariver")

['Aww', "Aon't", "ae'll", 'UUUU', 'and', 'IM', 'A9A', 'ae-l', 'ee', 'ef', 'aruck-ariver']


In [63]:
df_data['words_starting_with_vowel'] = df_data['TEXT'].apply(lambda x: findWordsStartingWithVowel(x))

In [64]:
df_data['words_starting_with_vowel'].sum()

14177

In [90]:
df_data.groupby(['LABEL'])['words_starting_with_vowel'].sum()

LABEL
0    6989
1    7188
Name: words_starting_with_vowel, dtype: int64

In [ ]:
def findWordsStartingWithConsonant(sentence):
  return len(re.findall("\\s([bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ][\w|À-Ö|Ø-ö|ø-ÿ|'|-]*)\\b", sentence))

In [ ]:
findWordsStartingWithConsonant("Aww Gi...Aon't worry:ae'll \n vote for UUUU and IM. A9A ae-l ee:ef aruck-ariver")

4

In [ ]:
df_data['words_starting_with_consonant'] = df_data['TEXT'].apply(lambda x: findWordsStartingWithConsonant(x))

In [ ]:
df_data['words_starting_with_consonant'].sum()

37196

## c. Lowercase the text and report the number of unique tokens present before and after lower casing.

Unique Tokens Before Splitting

In [105]:
def get_unique_tokens(tokens):
    unique_tokens = set()
    for token_list in tokens:
        for token in token_list:
            unique_tokens.add(token)
    return len(unique_tokens)

In [106]:
tokens = df_data['tokens'].to_list()

In [107]:
get_unique_tokens(tokens)

12612

In [108]:
def lowercase(text):
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    uppercase = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    for i in range(26):
        text = re.sub(uppercase[i], lowercase[i], text)
    return text

In [109]:
df_data['lowercase_text'] = df_data['TEXT'].apply(lambda x: lowercase(x))

In [110]:
df_data['lowercase_tokens'] = df_data['lowercase_text'].apply(lambda x: findTokens(x))

In [111]:
lowercase_tokens = df_data['lowercase_tokens'].to_list()

In [112]:
get_unique_tokens(lowercase_tokens)

11015

## d. Count and list all the usernames.

Basic Rules and Assumptions:

- According to [Twitter Guidelines](https://help.twitter.com/en/managing-your-account/twitter-username-rules)
  - Your username cannot be longer than 15 characters. Your name can be longer (50 characters) or shorter than 4 characters, but usernames are kept shorter for the sake of ease.
  - A username can only contain alphanumeric characters (letters A-Z, numbers 0-9) with the exception of underscores, as noted above. Check to make sure your desired username doesn't contain any symbols, dashes, or spaces.
  - *Optional Rule to Spot Users* - Usernames containing the words Twitter or Admin cannot be claimed. No account names can contain Twitter or Admin unless they are official Twitter accounts.

- Some experimentation with our Twitter Handle helped us reach the following conclusions:
  - @UserName can't be placed with other alphanumeric characters so abc@user 9@user are not valid and won't tag the user
  - X@UserName where X is a punctuation is valid but the following cases also don't allow tagging - 
    - @@xyz
    - _@xyz

Hence we propose regexes which actually find real tag matches instead of say a user just writing an @ somewhere in the tweet and it getting falsely matched as a tagged user when it really isn't

In [ ]:
s = ' @awaisnaseer I like Shezan Mangooo too!!! I '

In [ ]:
df_data['UserNames'] = df_data['TEXT'].apply(lambda x: findUsernames(x))
df_data['UserNamesCounts'] = df_data['TEXT'].apply(lambda x: findUsernameCount(x))

In [ ]:
df_data['UserNamesCounts'].sum()

2108

In [ ]:
usernames = df_data['UserNames'].to_numpy()
flattened_usernames = []
for i in usernames:
  flattened_usernames.extend(i)

In [ ]:
len(set(flattened_usernames))

2021

## e. Count and list all the urls

Basic Rules and Assumptions

- `http`, `https`, `www` common starters for URLs usually
- Manually investigating all sentences which contain `http` shows no false positives that is all occurences of `http` correspond to links
- Manually investigating all sentences which contain `www` shows lots of false positives due to words like `aww`
- On manual inspection we find no sentences which contain `https`
- According to [Twitter's official Blog](https://help.twitter.com/en/using-twitter/url-shortener), Twitter uses a URL-Shortener which converts links to the form `t.co` however there are no positive matches for this in our dataset. The only matches that arise are spurious matches in links like `blogspot.com`
- A URL maybe as simple as `www.xyz.abc` and as complex as `http://www.xyz.abc/efg` and can get even more complex by adding / to index more indepth into pages
- There are false positives that we encounter such as `Gi.don` or `worry.we` but they are both valid URLs as well as there can be custom domains by those names. We assume the domain will be atleast 2 characters long to account for `.me`, `.uk` etc.
- Numbers have been allowed as [only numbers can form valid URLs](https://stackoverflow.com/q/56804936/13858953)

In [263]:
from regexes import findURLs, findURLCount

In [264]:
findURLs("@katgkionis: FollowFriday is a blind 'Shout Out' recomendng ppl U like 2 follow.Goto http://twendz.com &amp; srch 4 followfriday 2 C! ")

['http://twendz.com']

In [265]:
findURLs("      https://www.google.c.uk.in 1.91 1.com com.1 http://www.Google.Co.Uk.in :://www.google.c.uk.in gogole.c www.google.c.uk the best http://apps.facebook.com/catbook/profile/view/620328 ( http://www.hakkastudy.in.th/) http://in.groups.yahoo.com/group/TheComicsClub/")

['https://www.google.c.uk.in',
 'Uk.in',
 'www.google.c.uk.in',
 'www.google.c.uk',
 'http://apps.facebook.com/catbook/profile/view/620328',
 'http://www.hakkastudy.in.th',
 'http://in.groups.yahoo.com/group/TheComicsClub']

In [259]:
df_data['URLs'] = df_data['TEXT'].apply(lambda x: findURLs(x))
df_data['URLCounts'] = df_data['TEXT'].apply(lambda x: findURLCount(x))

In [260]:
df_data['URLCounts'].sum()

205

In [261]:
for i in df_data[df_data['URLs'].apply(lambda x: len(x) > 0)]['URLs'].to_list():
    print(i)

['http://bit.ly/AEbs3']
['http://twitpic.com/3l589']
['http://blip.fm/~4lfcc']
['http://bit.ly/n4wL4']
['http://bit.ly/rwoHR']
['http://su.pr/1rXuPY']
['http://twitpic.com/6b03x']
['http://tinyurl.com/dk5p94']
['http://leo.lobato.org/Blipster']
['http://twitpic.com/4ijt4']
['http://bit.ly/nZZQV', 'http://bit.ly/etD3a']
['http://tinyurl.com/ncbmmo']
['http://dontkillspike.proboards.com']
['http://twitpic.com/6u8ht']
['ball.com/twitter']
['http://twendz.com']
['http://bit.ly/47etHn']
['http://twitpic.com/66zex']
['digg.com']
['http://twitpic.com/6fs89']
['http://bit.ly/i9lsr']
['http://www.myspace.com/xautomaticgirlx']
['www.musiqtone.com']
['http://twitpic.com/5exx2']
['http://plurk.com/p/z0xer']
['http://mypict.me/2dG2']
['http://blip.fm/~4kokb']
['http://twitpic.com/54r0g']
['http://apps.facebook.com/dogbook/profile/view/6391349']
['http://tinyurl.com/pboph6']
['http://is.gd/QAaz']
['www.paramore.net/shows']
['http://twitpic.com/680rp']
['http://tinyurl.com/mtq5u2']
['http://ustre.am/

## f. Count the number of tweets for each day of the week. Eg Mon: 58, Tues: 20, Wed...

In [ ]:
def getDay(sentence):
  return re.findall("Fri|Mon|Thu|Sun|Tue|Wed|Sat", sentence)[0]

In [ ]:
df_data['Day'] = df_data['DATE_TIME'].apply(lambda x: getDay(x))

In [ ]:
df_data['Day'].value_counts()

Sun    1328
Mon     872
Fri     864
Sat     417
Wed     299
Tue     286
Thu     221
Name: Day, dtype: int64

# B.

## a. Total number of occurrences of the given word and sentences containing that word.

In [ ]:
def re_find_word_in_sentence(word, sentence):
  return len(re.findall(f'\\b{word}\\b', sentence))

In [ ]:
def find_word_counts(df_data, word, class_label):
  filtered_df = df_data[df_data['LABEL'] == class_label]
  filtered_df['Counts'] = filtered_df['TEXT'].apply(lambda x: re_find_word_in_sentence(word, x))
  total_occurence = sum(filtered_df['Counts'])
  sentences_containing_word = filtered_df[filtered_df['Counts'] > 0]['TEXT'].to_list()
  return total_occurence, sentences_containing_word

In [ ]:
total_occurence, sentences_containing_word = find_word_counts(df_data, 'i', 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
total_occurence

450

In [ ]:
sentences_containing_word[:5]

["@buckhollywood I Cant Watch That i'm in the UK  Can you tell me what its about? Please x",
 '@mykiaisosm omj ur bad and mean i should not have meet u in 2nd grade even thouggh we hated each other i should have stayed like that ',
 ' i missed the game',
 "dr office... hopefully finding out why i've been so sick  it's so hard to keep my eyes open",
 'i feel like death...my next investment?going to the spa! i need a new body that can function  (via @IngaDurgin)i herd sleep is good 4 dat']

## b. Number of sentences starting with the given word.

In [ ]:
def re_find_sentence_starting_with_word(word, sentence):
  if re.findall(f'^\s*{word}\\b', sentence):
    return 1
  return 0

In [ ]:
re_find_sentence_starting_with_word("i", " am i "), re_find_sentence_starting_with_word("i", " i am i ")

(0, 1)

In [ ]:
def find_all_sentences_starting_with_word(df_data, word, class_label):
  filtered_df = df_data[df_data['LABEL'] == class_label]
  filtered_df['Counts'] = filtered_df['TEXT'].apply(lambda x: re_find_sentence_starting_with_word(word, x))
  total_occurence = sum(filtered_df['Counts'])
  sentences_starting_with_word = filtered_df[filtered_df['Counts'] == 1]['TEXT'].to_list()
  return total_occurence, sentences_starting_with_word

In [ ]:
total_occurence, sentences_starting_with_word = find_all_sentences_starting_with_word(df_data, 'i', 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
total_occurence, sentences_starting_with_word

(2,
 [' i missed the game',
  " i don't know, darlin. I'm so sorry. I wish i had what you need to make everything right."])

## c. Number of sentences ending with the given word.

In [ ]:
def re_find_sentence_ending_with_word(word, sentence):
  if re.findall(f'\\b{word}\s*$', sentence):
    return 1
  return 0

In [ ]:
re_find_sentence_ending_with_word("i", " am"), re_find_sentence_ending_with_word("i", " i am i")

(0, 1)

In [ ]:
def find_all_sentences_ending_with_word(df_data, word, class_label):
  filtered_df = df_data[df_data['LABEL'] == class_label]
  filtered_df['Counts'] = filtered_df['TEXT'].apply(lambda x: re_find_sentence_ending_with_word(word, x))
  total_occurence = sum(filtered_df['Counts'])
  sentences_ending_with_word = filtered_df[filtered_df['Counts'] == 1]['TEXT'].to_list()
  return total_occurence, sentences_ending_with_word

In [ ]:
total_occurence, sentences_ending_with_word = find_all_sentences_ending_with_word(df_data, 'scared', 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
total_occurence, sentences_ending_with_word

(1, ['About to get threaded and scared '])